In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [11]:
ib.disconnect()

In [2]:
from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *

In [3]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)

history = '6 Y'
freq = '1 day'
side ='Trades'

In [51]:
# Pull data from ADRs in "Asian_ADRs.csv"
adr_symbol = "ATHE"
adr_exchange = 'SMART'
adr_currency = "USD"

underlying_symbol = 'ATH'
underlying_exchange = 'SMART'
underlying_currency = 'AUD'
underlying_country = "Australia"

folder_name = underlying_country + "/" + adr_symbol + "_" + underlying_symbol + "/"

In [52]:
adr_contract = Contract(symbol = adr_symbol, secType = 'STK', exchange = adr_exchange, currency = adr_currency)
ib.qualifyContracts(adr_contract)
adr_df = get_data(adr_contract, history, freq, side)
print(adr_df.head(3))
adr_df.to_csv(folder_name + "adr.csv")

         date  open  high   low  close  volume   average  barCount
0  2016-03-24  3.40  3.40  3.01   3.05     543  3.151305       188
1  2016-03-28  3.04  3.05  2.90   2.99     487  2.965910       149
2  2016-03-29  3.00  3.05  2.80   2.84     316  2.902850       125


In [53]:
underlying_contract = Contract(symbol = underlying_symbol, secType = 'STK', exchange = underlying_exchange, currency = underlying_currency)
ib.qualifyContracts(underlying_contract)
underlying_df = get_data(underlying_contract, history, freq, side)
print(underlying_df.head(3))
underlying_df.to_csv(folder_name + "underlying.csv")

         date   open   high   low  close  volume  average  barCount
0  2015-04-17  0.140  0.145  0.14   0.14  127888  0.14005        19
1  2015-04-20  0.145  0.145  0.14   0.14   70030  0.14115        49
2  2015-04-21  0.142  0.145  0.14   0.14   67136  0.14435        28


In [4]:
def AUS_opening(date_now):
#     aus_dst = {
#         2021: (date(2021, 4, 4), date(2021, 10, 3)),
#         2020: (date(2020, 4, 5), date(2020, 10, 4)),
#         2019: (date(2019, 4, 7), date(2019, 10, 6)),
#         2018: (date(2018, 4, 1), date(2018, 10, 7)),
#         2017: (date(2017, 4, 2), date(2017, 10, 1)),
#         2016: (date(2016, 4, 3), date(2016, 10, 2)),
#         2015: (date(2015, 4, 5), date(2015, 10, 4)),
#     }
    
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    
    us_start, us_end = us_dst[date_now.year]
    if us_start < date_now and date_now < us_end:
        return datetime.combine(date_now,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_now, time(10,0)) - timedelta(hours = 15)

def HK_opening(date_now):
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_now, time(9,30)) - timedelta(hours = 13)
    
def JPN_opening(date_now):
    us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    }
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)
    

In [160]:
date = datetime.now()

print(AUS_opening(date))
print(HK_opening(date))
print(JPN_opening(date))

2021-04-15 20:00:00
2021-04-15 21:30:00
2021-04-15 20:00:00


In [5]:
forex_pairs = [["AUD",  "USD", "USD"],["USD", "HKD", "JPY",]]
opening_hours = [AUS_opening, HK_opening, JPN_opening]
df_dict = {}
forex_contracts = []
history = '300 S'
freq = '1 min'
side ='BID_ASK'

In [6]:
start = datetime.now()

close_date = datetime(2015,12,30,9,30)

# Initialize Dictionary to store dataframes and initialize all the contracts
for i in range(3):
    currency1 = forex_pairs[0][i]
    secType = "CASH"
    currency2 = forex_pairs[1][i]
    exchange = "IDEALPRO"
    
    df_dict[currency1+"."+currency2] = []
    
    forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
    ib.qualifyContracts(forex_contract)
    forex_contracts.append(forex_contract)
    
# Iterate over the datesace
while close_date < datetime.now():
    
    if close_date.weekday() >5: 
        close_date += timedelta(days = 1)
        continue
    
    for i in range(3):
        currency1 = forex_pairs[0][i]
        currency2 = forex_pairs[1][i]
        contract = forex_contracts[i]
        
        # NYSE Opening 
        enddate = close_date + timedelta(minutes = 2)
        forex_df_1 = get_data(contract, history, freq, side, enddate)
        df_dict[currency1+"."+currency2].append(forex_df_1)
        
        # Other Exchange Opening
        enddate = opening_hours[i](close_date) + timedelta(minutes = 2)
        forex_df_2 = get_data(contract, history, freq, side, enddate)
        df_dict[currency1+"."+currency2].append(forex_df_2)
        
    close_date += timedelta(days = 1)
    
end = datetime.now()
time_taken = end-start
print(time_taken)

Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: usfarm; cashhmds; ushmds; secdefil.
Peer closed connection


ConnectionError: Socket disconnect

In [ ]:
df_list = []
folder_name = "Forex/"
for i in range(3):
    currency1 = forex_pairs[0][i]
    currency2 = forex_pairs[1][i]
    df_forex = pd.concat(df_dict[currency1+"."+currency2], ignore_index = True,verify_integrity = True).sort_values(by = "date")
    df_forex.to_csv(folder_name + currency1 + "_" + currency2 + "_minute.csv")
    df_list.append(df_forex)
    print(df_forex.head(10))

In [ ]:
history = '180 S'
freq = '1 min'
side ='BID_ASK'

currency1 = forex_pairs[0][0]
secType = "CASH"
currency2 = forex_pairs[1][0]
exchange = "IDEALPRO"

enddate = datetime(2016, 1, 5)

# folder_name = "Forex/"

In [138]:
forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
ib.qualifyContracts(forex_contract)
forex_df = get_data(forex_contract, history, freq, side, enddate)
# print(forex_df.head(3))
# forex_df.to_csv(folder_name + currency1 + "_" + currency2 + ".csv")
# df_dict[currency1+"."+currency2].concat(forex_df)

In [82]:
test = df_dict["AUD.USD"]

In [90]:
test[[timestamp.time() == time(17,16) for timestamp in test["date"]]]

,date,open,high,low,close,volume,average,barCount
1,2015-12-30 17:16:00,0.72837,0.72848,0.72829,0.72844,-1,-1.0,-1


In [78]:
time(17,16).tzinfo